In [1]:
import copy
import logging
import numpy as np
import pandas as pd
from datetime import datetime,timedelta,date
import time
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoModel, get_linear_schedule_with_warmup, \
    default_data_collator, TrainingArguments, Trainer
from sentence_transformers import SentencesDataset, LoggingHandler, SentenceTransformer
from sklearn.metrics.pairwise import paired_cosine_distances, paired_euclidean_distances, paired_manhattan_distances
from scipy.stats import pearsonr, spearmanr
from transformers import AutoTokenizer
from transformers import BertConfig, BertTokenizer,BertTokenizerFast
from datasets import Dataset
from typing import List,Union,Dict
import tqdm
import re
import json
from dataclasses import dataclass
import os,time
os.chdir('/home/stops/Work_space/NLP_work/Med_assit_chatglm')
#os.environ['CUDA_VISIBLE_DEVICES']='0'

from db_config_taiyi import DB ## load data from pgsql

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO,format='%(asctime)s - %(levelname)s - %(message)s')
logger.info('Starting')

"""
1.采用小太医模板数据，测试Qwen-14b模型训练迁移能力。-231218
2.

"""


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/stops/miniconda3/envs/LLM_env/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /home/stops/miniconda3/envs/LLM_env/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


2023-12-18 15:27:01,878 - INFO : Starting


' \n1.采用小太医模板数据，测试Qwen-14b模型训练迁移能力。-231218\n2.\n\n'

In [2]:

init_prompt=["模拟问诊,你的角色是小太医医生，有丰富的医疗问诊知识和经验。"]
print('init_prompt: ',len(init_prompt),init_prompt[0])

init_output=["您好，我是小太医，有什么可以帮到您的？"]
print('init_output: ',len(init_output))



init_prompt:  1 模拟问诊,你的角色是小太医医生，有丰富的医疗问诊知识和经验。
init_output:  1


In [3]:
tokenizer_file="/home/stops/Work_space/NLP_models/Qwen-14B-Chat/"  ## Baichuan-V2
tokenizer=AutoTokenizer.from_pretrained(tokenizer_file,trust_remote_code=True)
logger.info(f"load data file: {tokenizer_file}")

test_text="我是小太医"
test_input_ids=tokenizer.encode(test_text)
print("test_input_ids: ",test_input_ids)
print(tokenizer.convert_ids_to_tokens(test_input_ids))

test_input_ids2=tokenizer.encode(test_text,add_special_tokens=True)
print("test_input_ids with special : ",test_input_ids2)

for sub_key ,sub_v in tokenizer.special_tokens_map.items():
    print(sub_key ,sub_v,tokenizer.convert_tokens_to_ids(sub_v))





2023-12-18 15:27:54,849 - INFO : load data file: /home/stops/Work_space/NLP_models/Qwen-14B-Chat/


test_input_ids:  [104198, 30709, 99222, 99286]
[b'\xe6\x88\x91\xe6\x98\xaf', b'\xe5\xb0\x8f', b'\xe5\xa4\xaa', b'\xe5\x8c\xbb']
test_input_ids with special :  [104198, 30709, 99222, 99286]


In [20]:
def show_df(df):
    print(df.shape)
    print(df.head(2))
    print(df.isnull().sum())

####一个对话字符串分割为角色，对话内容list
def convert_conv_to_role_utterance(conversation:str,split_symbol="[:：]"):
    ## [:：], ==>(前面有点)
    role_list_pre=re.findall('(?:患者|医生){}'.format(split_symbol),conversation)
    role_list=[re.sub('{}'.format(split_symbol),'',item) for item in role_list_pre]
    utterance_list_pre=re.split('(?:患者|医生){}'.format(split_symbol),conversation)
    text_list=[item.strip() for item in utterance_list_pre if  item!='' ]
    assert len(role_list)==len(role_list), '数目不等'
    return role_list,text_list


test_role_kw=['醫生','医生','顾问','D','牙医','医','医师','机器','Doctor','管家','AI','大夫','YS','Dr','Bot','H']
def convert_stand_role(role_list):
    doctor_role_kw=['bot','主任','专家','助手','Medical','系统','醫生','医生','顾问','D','牙医','医','医师','机器','Doctor','管家','AI','大夫','YS','Dr','Bot']
    stand_role_list=[]
    for sub_role in role_list:
        sub_doc_bool=[1 if item in sub_role  else 0 for item in doctor_role_kw]
        if sum(sub_doc_bool)>0:
            stand_role_list.append('医生')
        else:
            stand_role_list.append('患者')
    return stand_role_list

print(convert_stand_role(test_role_kw))


#### 检查近邻相同角色

def detect_neighborhood_role(role_list:List):
    pre_role=role_list[0]
    idx=0
    for sub_role in role_list[1:]:
        idx+=1
        if sub_role==pre_role:
            return True,(idx,sub_role)
        pre_role=sub_role
    return False,(None,None)

#test_role_l=['D','U','D','U','D','D']
test_role_l=['D','U','D','U','D','U']
test_res=detect_neighborhood_role(test_role_l)
print('test_res: ',test_res)


#### 相同角色合并
def concat_neighborhood_role_data(role_list:List,msg_list:List):
    assert len(role_list)==len(msg_list), 'not equal!!!  '+ msg_list[2]
    pre_role=role_list[0]
    idx=0
    keep_role_result=role_list[:1]
    keep_msg_result=msg_list[:1]
    for sub_role,sub_msg in zip(role_list[1:],msg_list[1:]):
        idx+=1
        if sub_role==pre_role:
            keep_msg_result[-1]=keep_msg_result[-1]+'\n'+sub_msg
        else:
            keep_role_result.append(sub_role)
            keep_msg_result.append(sub_msg)
        pre_role=sub_role
    return keep_role_result,keep_msg_result

test_role_l=['D','U','D','D','U','D','D']
test_msg_l=['D','U','D','D','U','D','D']

#test_role_l=['D','U','D','U','D','U']
#test_msg_l=['D','U','D','U','D','U']
print('len: ',len(test_role_l),len(test_msg_l))
test_rol_res,test_msg_res=concat_neighborhood_role_data(test_role_l,test_msg_l)
print('len: ',len(test_rol_res),len(test_msg_res))
print('test_rol_res: ',test_rol_res)
print('test_msg_res: ',test_msg_res)


#### 规则判断是否为疑问句
doc_query_kw_list = ['是不是', '？', '\\?', '呢', '多少', '吗', '怎样', '怎么', '是否', '能否', '有无', '有没有'
            , '什么', '哪里', '哪些', '咋样', '如何', '区别', '那些', '哪', '多长', '多久', '多少', '多高']
kw_expression = '|'.join(doc_query_kw_list)
quest_compile = re.compile('{}'.format(kw_expression))

def query_bool(text:str):
    if len(quest_compile.findall(text)) > 0:
        return True
    else:
        return False

test='请问您还有其他问题需要咨询吗'
print('test: ',test)
print(query_bool(test))

#### 规则判断是否为疑问句
diagnosis_kw_list = ['根据','建议','避免','推荐','采取以下措施','如果症状']
diagnosis_kw_expression = '|'.join(diagnosis_kw_list)
diagnosis_compile = re.compile('{}'.format(diagnosis_kw_expression))
def diagnosis_bool(text:str):
    if len(diagnosis_compile.findall(text)) > 0:
        return True
    else:
        return False

test='非常感谢，我已经查看了您的检查报告单。根据您的症状，您可能患有一种病毒性感染病。建议您继续使用对乙酰氨基酚退热。另外，你应该加强自身抵抗力，多休息、多饮水，同时注意科学饮食，增加营养，提高身体免疫力。'
print('test: ',test)
print(diagnosis_bool(test))

def doc_end_kw_bool(text:str):
    for sub_kw in ["早日康复","客气"]:
        if sub_kw in text:
            return True
    if "如果您还有其他" in text and len(text)<=20:
        return True
    else:
        return False

print('****'*3)
print(doc_end_kw_bool("如果您还有其他问题可以随时联系我哦~"))
print(doc_end_kw_bool("我右手和右腿还是麻木"))
print(doc_end_kw_bool("右手和右腿还是麻木?"))
print('****'*3)

def sentence_with_punk_book(text:str):
    text=text.strip()
    if text[-1] in ['。','!','！','.','，',',','，']:
        return True
    else:
        return False

print(sentence_with_punk_book("稳定情绪，避免过度紧张、压力过大，保证充足的睡眠。"))


## 检查结果
media_kw_list = ['照片','图片', '报告', '图像','检查结果']
media_kw_expression = '|'.join(media_kw_list)
media_kw_compile = re.compile('{}'.format(media_kw_expression))

def detect_media_text(text:str):
    if len(media_kw_compile.findall(text)) > 0:
        return True
    else:
        return False

print(detect_media_text("麻烦上传一下照片"))

def process_doctor_sorry(text):
    if isinstance(text,str):
        result=re.sub('.*很抱歉.*?[.。,，]{1}','',text)
        return result
    if isinstance(text,List):
        result_list=[process_doctor_sorry(item) for item in text]
        return result_list

test="我很抱歉听到这个消息,请问持续多久了?"
print('test: ',test)
print(process_doctor_sorry(test))

test_list=["我很抱歉听到这个消息,请问持续多久了?","你好，很抱歉听到这个消息。","你好，听到这个消息。"]
print('test_list: ',test_list)
print(process_doctor_sorry(test_list))

['医生', '医生', '医生', '医生', '医生', '医生', '医生', '医生', '医生', '医生', '医生', '医生', '医生', '医生', '医生', '患者']
test_res:  (False, (None, None))
len:  7 7
len:  5 5
test_rol_res:  ['D', 'U', 'D', 'U', 'D']
test_msg_res:  ['D', 'U', 'D\nD', 'U', 'D\nD']
test:  请问您还有其他问题需要咨询吗
True
test:  非常感谢，我已经查看了您的检查报告单。根据您的症状，您可能患有一种病毒性感染病。建议您继续使用对乙酰氨基酚退热。另外，你应该加强自身抵抗力，多休息、多饮水，同时注意科学饮食，增加营养，提高身体免疫力。
True
************
True
False
False
************
True
True
test:  我很抱歉听到这个消息,请问持续多久了?
请问持续多久了?
test_list:  ['我很抱歉听到这个消息,请问持续多久了?', '你好，很抱歉听到这个消息。', '你好，听到这个消息。']
['请问持续多久了?', '', '你好，听到这个消息。']


In [13]:

def main_process_one_conv_pair(sub_idx,sub_stand_role_result:List,sub_conv_detail_list_pre:List,verbose=False):
    except_dis_detect_nums=0
    Media_conv_num=0
    delete_mode=False ## 控制是否删除当前conversation
    ############
    ## p-1:转为标准用户，医生角色
    ## 患者，医生
    #############
    sub_stand_role_result=convert_stand_role(sub_stand_role_result)
    #############
    ## p-4:剔除"我是你的医生"
    ## "你好，很抱歉听到这个消息。"
    #############
    sub_conv_detail_list=[]
    for sub_i,sub_dat in enumerate(zip(sub_conv_detail_list_pre,sub_stand_role_result)):
        sub_item,sub_rol=sub_dat
        if sub_rol=='医生' and sub_i<4:
            sub_doc_detect=re.findall('我是[^.。,，?!？！]*医生[，.。]',sub_item)
            if sub_doc_detect:
                if len(sub_doc_detect)==1:
                    #print('sub_doc_detect: ',sub_doc_detect)
                    sub_doc_detect_str=sub_doc_detect[0]
                    sub_item=sub_item.replace(sub_doc_detect_str,'')
                else:
                    print('*********')
                    print('**  >2 '+'我是你的医生: '+sub_item)
        sub_conv_detail_list.append(sub_item)
    ## 处理美式道歉-sorry
    sub_conv_detail_list=process_doctor_sorry(sub_conv_detail_list)
    #############
    ## p-5:检查近邻相同角色，['D','U','D','U','D','D']
    #############
    sub_det_bool,sub_det_res=detect_neighborhood_role(sub_stand_role_result)
    if sub_det_bool :
        print('    neighborhood : ',sub_idx,"|".join(sub_conv_detail_list)[:30])
        print('                 : ',sub_idx,"|".join(sub_stand_role_result))
        sub_stand_role_result,sub_conv_detail_list=concat_neighborhood_role_data(sub_stand_role_result,sub_conv_detail_list)
    if verbose:
        print('    sub_stand_role_result num: ',len(sub_stand_role_result))
        print(sub_stand_role_result)
        print('    sub_conv_detail_list  num: ',len(sub_conv_detail_list))
        print(sub_conv_detail_list)
    #print(sub_conv_detail_list)
    assert len(sub_conv_detail_list)==len(sub_stand_role_result) , '数目不等 :'+str(len(sub_stand_role_result))+' '+str(len(sub_conv_detail_list))
    ##############
    ## p-8:替换医生角色特殊语句
    ## 剔除"我们"，"我"
    ## 最后3轮，患者角色，长度超过20，查看异常，是否出先角色位置异常情况。
    ##############
    sub_conv_detail_list_8=[]
    sub_loops=len(sub_stand_role_result)
    sub_loop_i=0
    for sub_role,sub_text in zip(sub_stand_role_result,sub_conv_detail_list):
        sub_loop_i+=1
        if sub_role=="医生" and diagnosis_bool(sub_text):
            sub_text=sub_text.replace("我们","")
            sub_text=sub_text.replace("我","")
            sub_text=sub_text.replace("线上医生","")
        if (sub_loop_i>=sub_loops-3) and sub_loop_i>=6:
            if sub_role=="患者"  and len(sub_text)>90:
                print("Be Careful: role swrap",str(sub_idx),sub_text[:20])
        sub_conv_detail_list_8.append(sub_text)

    assert len(sub_conv_detail_list_8)==len(sub_stand_role_result) , '数目不等[2] '+'  '+str(len(sub_stand_role_result))+' '+str(len(sub_conv_detail_list))+' '+ sub_session_cont[:20]
    return sub_stand_role_result,sub_conv_detail_list_8

def process_last_doctor_add_eval_require(sub_stand_role_result,sub_conv_detail_list,verbose=False):
    add_eval_kw_string="本次服务对您是否有帮助，请进行评价，谢谢。"
    ##############
    ## p-7:结尾句处理
    ## "本次服务对您是否有帮助，请进行评价，谢谢。"
    ## 重点看医生角色的最后3个语句。重点出现[根据，建议，避免，推荐],只添加一次
    ## add_words_mode=False
    ## for sub_text in last_words:
    ##    if diagnosis_bool(sub_text) and not query_bool(sub_text) and not add_words_mode:
    ##        run_add_words(sub_text)
    ##        add_words_mode=True
    ## 其他情况不做处理。
    ##############
    ## 寻找医生最后3个位置索引
    sub_last_doc_idx_list=[]
    if sub_stand_role_result[-1]=="医生":
        sub_last_doc_idx_list=[len(sub_stand_role_result)-1]
        if len(sub_stand_role_result)>3:
            sub_last_doc_idx_list.insert(0,len(sub_stand_role_result)-3)
        if len(sub_stand_role_result)>5:
            sub_last_doc_idx_list.insert(0,len(sub_stand_role_result)-5)
    else:
        sub_last_doc_idx_list=[len(sub_stand_role_result)-2]
        if len(sub_stand_role_result)>4:
            sub_last_doc_idx_list.insert(0,len(sub_stand_role_result)-4)
        if len(sub_stand_role_result)>6:
            sub_last_doc_idx_list.insert(0,len(sub_stand_role_result)-6)
    if verbose:
        print("sub_last_doc_idx_list: ",len(sub_stand_role_result))
        print(sub_stand_role_result[-4:])
        print(sub_last_doc_idx_list)
    ## 最后3个医生角色对应语句
    sub_last_doc_cont_list=[sub_conv_detail_list[idx] for idx in sub_last_doc_idx_list]
    ## 保存处理后的结果，最后按索引替换结果
    sub_last_doc_cont_result_list=copy.deepcopy(sub_last_doc_cont_list)
    if verbose:
        print("before process sub_last_doc_cont_result_list: ")
        print(sub_last_doc_cont_result_list)
    sub_add_words_mode=False
    sub_last_doc_add_text=''
    sub_last_doc_add_idx=None
    for sub_last_idx,sub_last_doc_text in zip(sub_last_doc_idx_list,sub_last_doc_cont_result_list):
        sub_diagnosis_bool=diagnosis_bool(sub_last_doc_text)
        sub_query_bool=query_bool(sub_last_doc_text)
        if sub_diagnosis_bool and (not sub_query_bool) and (not sub_add_words_mode):
            if sentence_with_punk_book(sub_last_doc_text):
                sub_last_doc_add_text=sub_last_doc_text+add_eval_kw_string
            else:
                sub_last_doc_add_text=sub_last_doc_text+"。"+add_eval_kw_string
            sub_last_doc_add_idx=sub_last_idx
            sub_add_words_mode=True
    ## 按索引替换结果
    if sub_add_words_mode and len(sub_last_doc_add_text)>0 and sub_last_doc_add_idx:
        sub_conv_detail_list[sub_last_doc_add_idx]=sub_last_doc_add_text
    ## 判断本次对话是否添加评价
    check_last_doc_cont_list=[sub_conv_detail_list[idx] for idx in sub_last_doc_idx_list]
    check_eval_bool=sum([1 if add_eval_kw_string in item  else 0 for item in check_last_doc_cont_list])
    if check_eval_bool>0:
        if verbose:
            print("此处有评价")
        ## 说明最后有评价语句
        pass
    else:
        if verbose:
            print("此处无评价")
        check_last_doc_cont_len_l=[len(sub_conv_detail_list[idx]) for idx in sub_last_doc_idx_list]
        check_last_doc_cont_max_len=max(check_last_doc_cont_len_l)## 后三个中最大长度值
        check_last_max_len_idx=check_last_doc_cont_len_l.index(check_last_doc_cont_max_len)#### 后三个中最大长度值的对应的索引
        check_last_max_len_idx_in_all_seq=sub_last_doc_idx_list[check_last_max_len_idx]## 后三个中最大长度值的对应的索引==>全序列中的索引
        sub_max_len_cont_string=sub_conv_detail_list[check_last_max_len_idx_in_all_seq]
        if verbose:
            print("最长长度句： ",sub_max_len_cont_string)
        sub_check_query_bool=query_bool(sub_max_len_cont_string)
        if not sub_check_query_bool:
            if verbose:
                print("正常按最长长度处理，且无疑问句： ",sub_max_len_cont_string)
            if sentence_with_punk_book(sub_max_len_cont_string):
                sub_conv_detail_list[check_last_max_len_idx_in_all_seq]=sub_max_len_cont_string+add_eval_kw_string
            else:
                sub_conv_detail_list[check_last_max_len_idx_in_all_seq]=sub_max_len_cont_string+"。"+add_eval_kw_string
        else:
            ##虽然在最大长度位置，但是有疑问句，不做处理
            pass
    if verbose:
        print("after process sub_last_doc_cont_result_list: ")
        print(sub_last_doc_add_idx,sub_last_doc_add_text)
    return sub_stand_role_result,sub_conv_detail_list


In [22]:
def build_norm_conversation_history_data(conversation_role_result_list,conversation_pair_result_list,
                                         knowledge_mode=False,Symptom_text_list=None,med_knowledge_list=None):
    cot_prompt_template="""你的角色是小太医专业医生，你的身份是是由上海杉泰健康科技有限公司于研发的AI预诊助手。{symptom}症状医学相关知识如下:\n{text}\n任务：结合上面医学知识，进行问诊。"""
    assert len(conversation_role_result_list)==len(conversation_pair_result_list) , "数目不等！！！"
    run_nums=len(conversation_role_result_list)
    verbose=False
    conversation_history_result_list=[]
    test_nums=3
    proc_start_result_list=[]
    #for sub_idx,sub_data in enumerate(zip(conversation_role_result_list[:test_nums],conversation_pair_result_list[:test_nums])):
    for sub_idx,sub_data in enumerate(zip(conversation_role_result_list,conversation_pair_result_list)):
        if sub_idx%500==0:
            if test_nums and  verbose:
                logger.info("run steps1: {}, finished: {:.2%}".format(sub_idx,sub_idx/test_nums))
            else:
                logger.info("run steps2: {}, finished: {:.2%}".format(sub_idx,sub_idx/run_nums))
        sub_role_data=sub_data[0]
        sub_child_sess_data=sub_data[1]
        if len(sub_role_data)<=2:
            print('len(sub_role_data)<=2: ')
            print('sub_child_sess_data: ',sub_child_sess_data[:2])
        if verbose:
            print('***'*12)
            print('    Raw sub_child_sess_data nums: ',len(sub_child_sess_data))
        """
        从第一轮开始预测,如果首个角色是医生，且"有什么症状",则舍去如果最后一个角色是用户患者，则舍去，因为要预测医生问诊的内容。
        """
        ## 从头进行构造数据
        sub_child_sess_data_proc=copy.deepcopy(sub_child_sess_data)
        sub_role_data_proc=copy.deepcopy(sub_role_data)
        ##############
        ## 起始句，采用医学知识
        ## sub_init_prompt,sub_init_output
        ##############
        if knowledge_mode:
            sub_init_symptom=Symptom_text_list[sub_idx]
            sub_init_med_knowledge=med_knowledge_list[sub_idx]
            sub_init_prompt=cot_prompt_template.format(symptom=sub_init_symptom,text=sub_init_med_knowledge)
        else:
            sub_init_prompt=np.random.choice(init_prompt,1)[0]

        if verbose:
            print('      sub_init_prompt     :sub_init_prompt')
        sub_init_output=np.random.choice(init_output,1)[0]
        if verbose:
                print('    step-0      : ',len(sub_role_data_proc),len(sub_child_sess_data_proc))
        if verbose:
            #print('sub_init_prompt: ',sub_init_prompt)
            #print('sub_init_output: ',sub_init_output)
            pass
        ############################################################
        ## step-1 如果首个角色是医生，且"有什么症状",则舍去
        ## 最终统一为，第一轮为开场句，不做预测
        ############################################################
        if sub_role_data[0]=='医生':
            if '什么' in sub_child_sess_data[0] or '哪些' in sub_child_sess_data[0] or '怎么' in sub_child_sess_data[0]:
                ## 增加角色位场景。内容首先替换标准问候语，同时追加场景词
                ## role: ['场景','医生',||'患者','医生'...], sess: ['你的角色医生...','我是小太医',...]
                sub_role_data_proc.insert(0,'场景')
                sub_child_sess_data_proc[0]=sub_init_output
                sub_child_sess_data_proc.insert(0,sub_init_prompt)
                if verbose:
                    print('    step-1-1: ',len(sub_role_data_proc),len(sub_child_sess_data_proc),sub_child_sess_data_proc[:2])
            else :
                ###########
                ## 增加角色位场景，同时前缀增加"这里是太医管家，"
                #########
                ## role: ['场景','医生','患者'，||(正式预测)'医生'...], sess: ['你的角色医生...','我是小太医，你头痛2天了','怎么办呀'，||(正式预测)'连续痛吗？'..]
                sub_role_data_proc.insert(0,'场景')
                prefix_doc_utterance='我是小太医，'##后面是医生描述的症状，如'我是小太医，你头痛2天了'
                sub_child_sess_data_proc[0]=prefix_doc_utterance+sub_child_sess_data_proc[0]
                sub_child_sess_data_proc.insert(0,sub_init_prompt)
                if verbose:
                    print('    step-1-2: ',len(sub_role_data_proc),len(sub_child_sess_data_proc),sub_child_sess_data_proc[:2])
        ## 只有首句进行操作
        else:
            sub_role_data_proc.insert(0,'医生')
            sub_child_sess_data_proc.insert(0,sub_init_output)
            if verbose:
                print('    step-1-3-1: ',len(sub_role_data_proc),len(sub_child_sess_data_proc))
            sub_role_data_proc.insert(0,'场景')
            sub_child_sess_data_proc.insert(0,sub_init_prompt)
            if verbose:
                print('    step-1-3-2: ',len(sub_role_data_proc),len(sub_child_sess_data_proc))
                print('    step-1-3: ',len(sub_role_data_proc),len(sub_child_sess_data_proc),sub_child_sess_data_proc[:2])
        if verbose:
            print('    add data sub_child_sess_data nums: ',len(sub_child_sess_data_proc))
            print(sub_child_sess_data_proc)
        ## 到此构造成一个内容有提示开始，角色以场景开始的 数据对
        ##---------------------
        ############################################################
        ## step-2:如果最后一个角色是用户患者，则舍去，因为要预测医生问诊的内容。
        ############################################################
        if sub_role_data[-1]=='患者':
            sub_role_data_proc.pop(-1)
            sub_child_sess_data_proc.pop(-1)
            if verbose:
                print('    step-2: ',len(sub_role_data_proc),len(sub_child_sess_data_proc),sub_child_sess_data_proc[:2])
        if verbose:
            print('role nums: ',len(sub_role_data_proc))
            print('conv nums: ',len(sub_child_sess_data_proc))
        assert len(sub_role_data_proc)==len(sub_child_sess_data_proc) ,'对话数目不等于角色数量 idx:'+str(sub_idx)
        ## 构造会话组合对[[U,D],[U,D]....],因为要预测的D角色的内容
        sub_conv_pair_list=[]
        for idx,sub_utterance in enumerate(sub_child_sess_data_proc):
            if idx%2==0:
                sub_conv_pair_list.append([sub_utterance])
                if verbose:
                    print('USER ==> ',sub_utterance)
            else:
                sub_conv_pair_list[-1].append(sub_utterance)
                if verbose:
                    print('DOCTOR ==> ',sub_utterance)
        if verbose:
            print(' build conv nums: ',len(sub_conv_pair_list))
        ## 如果会话组合对为奇数,即[[U,D],[U,D]...[U,]]，则删除最后一个用内容
        sub_child_multi_sum_turns=sum([len(item) for item in sub_conv_pair_list])
        sub_child_multi_turns=len(sub_conv_pair_list)
        if sub_child_multi_sum_turns%2!=0:
            print('odd num ,sub_idx :',sub_idx,' turns: ',sub_child_multi_turns,sub_conv_pair_list[-1])
            sub_conv_pair_list=sub_conv_pair_list[:-1]
            sub_child_multi_turns=len(sub_conv_pair_list)
            print('                        final turns:',sub_child_multi_turns)
        ## 最终 训练数据对: sub_conv_pair_list, [[U,D],[U,D],...]
        ##########
        ## 从头进行构造History数据
        ##########
        sub_dialog_text_list=[]
        for sub_pair_text in sub_conv_pair_list:
            sub_user_text=sub_pair_text[0]
            sub_doctor_text=sub_pair_text[1]
            sub_dialog_text_list.append({"role":"user","text":sub_user_text})
            sub_dialog_text_list.append({"role":"assistant","text":sub_doctor_text})
        conversation_history_result_list.append(sub_dialog_text_list)
    logger.info(f"conversation_history_result_list: {len(conversation_history_result_list)}")
    return conversation_history_result_list



In [14]:
##################################
## Expl-Data-5 To-Symptom-template-True-Scenarios
##################################
symptom_template_true_scenarios_data_file="data/Top_Symptom_true_scenarios_template_231214.xlsx"


In [15]:
symptom_template_true_df_norm=pd.read_excel(symptom_template_true_scenarios_data_file,sheet_name="正常")
symptom_template_true_df_drug=pd.read_excel(symptom_template_true_scenarios_data_file,sheet_name="药物")
symptom_template_true_df_pic1=pd.read_excel(symptom_template_true_scenarios_data_file,sheet_name="便秘图片")
symptom_template_true_df_pic2=pd.read_excel(symptom_template_true_scenarios_data_file,sheet_name="咽喉图片")
symptom_template_true_df_pic3=pd.read_excel(symptom_template_true_scenarios_data_file,sheet_name="腹泻图片")

print("******"*6+"  1")
show_df(symptom_template_true_df_norm)
print("******"*6+"  2")
show_df(symptom_template_true_df_drug)
print("******"*6+"  3")
show_df(symptom_template_true_df_pic1)
print("******"*6+"  4")
show_df(symptom_template_true_df_pic2)
print("******"*6+"  5")
show_df(symptom_template_true_df_pic3)


************************************  1
(559, 2)
     order_id                                            content
0  23206461.0  患者: 您好，声带白斑好治疗吗,是的\n医生: 请问您发现声带白斑有多久了呢？有相关报告吗？...
1  23206784.0  患者: 昨天腰扭伤在家怎么保守治疗\n医生: 不适的部位是在图中哪里呢？（28、左肩胛下，2...
order_id    4
content     0
dtype: int64
************************************  2
(182, 2)
   order_id                                            content
0  23197979  患者: 石大夫，我有点感冒吃什么药\n医生: 秦悦女47岁你自己的情况吗？感冒几天了？\n患...
1  23202979  患者: 喉咙痛\n医生: 请问症状出现有多久了呢\n患者: 昨晚刚刚这样\n医生: 请问您是...
order_id    0
content     0
dtype: int64
************************************  3
(24, 2)
   order_id                                            content
0  23032479  患者: 2岁宝宝便秘 能喝妈咪爱吗\n医生: 了解一下孩子的具体情况，请问便秘有多久了呢\n...
1  23038450  患者: 孩子最近总说肚子不舒服，有点拉肚子，吃了乳酸菌素片还是说不舒服,晚上睡觉趴着睡\n医...
order_id    0
content     0
dtype: int64
************************************  4
(37, 2)
   order_id                                            content
0  23049351  患者: 嗓子疼\n医生: 您好，咽痛几天了？\n患者: 我嗓子疼了两

In [16]:
symptom_template_true_df=pd.concat([symptom_template_true_df_norm,symptom_template_true_df_drug,symptom_template_true_df_pic1,
                                 symptom_template_true_df_pic2,symptom_template_true_df_pic3])
print(symptom_template_true_df.shape)

true_scenario_symptom_template_df=pd.concat([symptom_template_true_df,symptom_template_true_df.sample(frac=0.5)])
true_scenario_symptom_template_df=true_scenario_symptom_template_df.reset_index(drop=True)
print(true_scenario_symptom_template_df.shape)
print(true_scenario_symptom_template_df.head(2))
print(true_scenario_symptom_template_df.isnull().sum())
print(true_scenario_symptom_template_df['content'].duplicated().sum())


(820, 2)
(1230, 2)
     order_id                                            content
0  23206461.0  患者: 您好，声带白斑好治疗吗,是的\n医生: 请问您发现声带白斑有多久了呢？有相关报告吗？...
1  23206784.0  患者: 昨天腰扭伤在家怎么保守治疗\n医生: 不适的部位是在图中哪里呢？（28、左肩胛下，2...
order_id    6
content     0
dtype: int64
410


In [17]:
true_symptom_template_conv_pair_result_list=[]
true_symptom_template_conv_role_result_list=[]
max_output_len=[]
max_len_text=""
max_len=0
for sub_idx, sub_row in true_scenario_symptom_template_df.iterrows():
    sub_max_output_len=[]
    sub_conv=sub_row["content"]
    ## split_symbol="[:：]"
    sub_stand_role_result,sub_conv_detail_list=convert_conv_to_role_utterance(sub_conv)
    if pd.Series(sub_stand_role_result).value_counts().shape[0]!=2:
        print("sub_idx: ",sub_idx,"|".join(sub_stand_role_result))
    for sub_role,sub_text in zip(sub_stand_role_result,sub_conv_detail_list):
        if sub_role=='医生':
            sub_doc_len=len(sub_text)
            sub_max_output_len.append(sub_doc_len)
            if sub_doc_len>max_len:
                max_len=sub_doc_len
                max_len_text=sub_text
    max_output_len.extend(sub_max_output_len)
    true_symptom_template_conv_pair_result_list.append(sub_conv_detail_list)
    true_symptom_template_conv_role_result_list.append(sub_stand_role_result)

logger.info(f'true_symptom_template_conv_pair_result_list nums: {len(true_symptom_template_conv_pair_result_list)}')
logger.info(f'examples: {true_symptom_template_conv_pair_result_list[:2]}')

logger.info(f'true_symptom_template_conv_role_result_list nums: {len(true_symptom_template_conv_role_result_list)}')
logger.info(f'examples: {true_symptom_template_conv_role_result_list[:2]}')
logger.info(f'doctor text max length nums: {max_len}')
print("max_len_text: ",max_len_text)


2023-12-18 15:31:51,799 - INFO : true_symptom_template_conv_pair_result_list nums: 1230
2023-12-18 15:31:51,799 - INFO : examples: [['您好，声带白斑好治疗吗,是的', '请问您发现声带白斑有多久了呢？有相关报告吗？', '报告在家里', '请问有没有声音嘶哑、咽部异物感、声带活动受限等情况呢？', '我现在只能把情况跟你讲讲，晚上您要是有空我可以发给你', '可以的啊,我们晚上也有医生在的', '就是因为嗓子哑，还伴有咳嗽，这种情况从3月份阳了以后就开始了，但是我上个月才去医院做的喉镜才茶出来,查出来', '咳嗽目前有痰吗？', '我现在就是在家做雾化，每天2次，用的是人干扰素2a，地塞米松磷酸钠，和氯化钠，还有一个治疗食道反流的药,没痰', '您是否有哮喘、支气管炎、鼻窦炎、扁桃体炎、反流性食管炎病史?', '当时治疗一周以后复查的喉镜有所好转，现在还在继续，每天晚上起来声音会好一点，但只要一到晚上就会比较哑，而且，现在体重长的也比较快，从用药到现在长了8斤，我已经做雾化快4周的，这个发胖跟用地塞米松有关系吗？是激素造成的吧/', '一般来说，雾化的用药不会引起肥胖。\n根据您的情况，以下是饮食和治疗建议：\n治疗建议：\n根据您的描述，声带白斑可能是由于声带炎症引起的。建议继续进行医生建议的雾化治疗，同时配合药物治疗食道反流。如果嗓子哑、咳嗽等症状持续存在，建议进行进一步的喉镜检查以了解病情发展情况。\n饮食建议：\n饮食方面，需要注意避免辛辣、刺激性食物，如辣椒、生姜等，以免加重喉咙的不适感。建议多喝水，保持喉咙湿润，有助于缓解症状。\n运动建议：\n可以适量进行喉部运动，如喉咙放松操、喉咙振动操等，有助于改善声带状况。此外，避免大声喊叫、长时间使用嗓子，保护声带健康。'], ['昨天腰扭伤在家怎么保守治疗', '不适的部位是在图中哪里呢？（28、左肩胛下，29、右肩胛下，21、腰椎，30、左腰部，31、右腰部）您可以直接跟我说数字。', '21', '目前腰部活动有没有明显受限？有没有下肢的麻木无力酸痛？这边没有收到您的回复，可能是在忙，等您不忙了以后我们再沟通。', '对不起了，有点事没回复，活动不受限，下肢不麻木。就是穿衣

max_len_text:  根据您的情况，以下是饮食和治疗建议：
治疗建议：
1. 首先，继续服用疼痛科医生开的药物，按医生的用药指导进行用药。
2. 可以尝试在腰部疼痛的地方进行冷敷，每次冷敷15-20分钟，可以缓解疼痛和肿胀。
3. 避免剧烈运动和重物提起，保持适度的活动，避免长时间保持一个姿势，避免腰部过度用力。
4. 可以尝试进行腰部轻柔按摩，可以缓解肌肉紧张和疼痛。
5. 如果疼痛持续存在或者加重，建议及时复诊，以便进行进一步的评估和治疗。
饮食建议：
1. 保持均衡饮食，摄入足够的蛋白质、维生素和矿物质，促进伤口愈合和肌肉修复。
2. 增加摄入富含维生素C和维生素D的食物，如柑橘类水果、番茄、红椒、鱼类等，有助于骨骼健康。
3. 避免摄入过多的高脂肪、高糖和高盐食物，以免加重身体的炎症反应。
4. 保持充足的水分摄入，帮助维持身体的代谢功能和组织修复。
5. 如果遇到饮食上的问题或者需要特殊的饮食调整，建议咨询营养师的意见。
运动建议：
1. 在医生指导下进行适量的伸展运动，帮助放松腰部肌肉和提高腰部的灵活性。
2. 避免剧烈运动和高强度的活动，以免加重腰部损伤。
3. 如果有需要，可以咨询物理治疗师或康复医师，进行专业的康复训练，促进腰部功能的恢复和强化。
4. 注意保持正确的姿势，避免长时间保持一个姿势，尤其是长时间坐姿，可以适当站起活动一下。
5. 如果遇到运动上的问题或者需要特殊的运动指导，建议咨询专业的运动医学或康复医学机构的意见。


In [21]:
## D5 处理格式问题，如相邻角色，添加评价等
proc_true_symptom_template_conv_role_result_list=[]
proc_true_symptom_template_conv_pair_result_list=[]

sub_id=0
for sub_role_list,sub_text_list in zip(true_symptom_template_conv_role_result_list,true_symptom_template_conv_pair_result_list):
    sub_role_list1,sub_text_list1=main_process_one_conv_pair(sub_id,sub_role_list,sub_text_list)
    sub_role_list2,sub_text_list2=process_last_doctor_add_eval_require(sub_role_list1,sub_text_list1)
    proc_true_symptom_template_conv_role_result_list.append(sub_role_list2)
    proc_true_symptom_template_conv_pair_result_list.append(sub_text_list2)
    sub_id+=1

logger.info(f"proc_true_symptom_template_conv_role_result_list nums: {len(proc_true_symptom_template_conv_role_result_list)}")
logger.info(f"proc_true_symptom_template_conv_pair_result_list nums: {len(proc_true_symptom_template_conv_pair_result_list)}")


2023-12-18 15:32:20,668 - INFO : proc_true_symptom_template_conv_role_result_list nums: 1230
2023-12-18 15:32:20,668 - INFO : proc_true_symptom_template_conv_pair_result_list nums: 1230


Be Careful: role swrap 0 当时治疗一周以后复查的喉镜有所好转，现在
    neighborhood :  3 打鼾可以治疗吗|打鼾是可以治疗的。每个人的病因不同,所以治疗
                 :  3 患者|医生|患者|医生|患者|医生|患者|医生|患者|医生|患者|医生|医生
Be Careful: role swrap 9 没有这些你你说的以上症状，反正就是咳嗽，
Be Careful: role swrap 285 昨天中午在客户家吃的，吃了蒸红薯、米饭、
Be Careful: role swrap 576 只是做卧推动作的时候重量大一点的时候会明
Be Careful: role swrap 602 反正就是嗯，也没吃啥，有时候你喝个水了，
Be Careful: role swrap 686 39度，前两天白天晚上的咳嗽，晚上睡不好
Be Careful: role swrap 902 反正就是嗯，也没吃啥，有时候你喝个水了，
    neighborhood :  939 打鼾可以治疗吗|打鼾是可以治疗的。每个人的病因不同,所以治疗
                 :  939 患者|医生|患者|医生|患者|医生|患者|医生|患者|医生|患者|医生|医生
Be Careful: role swrap 1169 39度，前两天白天晚上的咳嗽，晚上睡不好


In [24]:

proc_true_symptom_template_conversation_history_data=build_norm_conversation_history_data(proc_true_symptom_template_conv_role_result_list,
                                    proc_true_symptom_template_conv_pair_result_list,
                                     knowledge_mode=False,Symptom_text_list=None,med_knowledge_list=None)

2023-12-18 15:34:00,698 - INFO : run steps2: 0, finished: 0.00%
2023-12-18 15:34:00,721 - INFO : run steps2: 500, finished: 40.65%
2023-12-18 15:34:00,745 - INFO : run steps2: 1000, finished: 81.30%
2023-12-18 15:34:00,756 - INFO : conversation_history_result_list: 1230


In [34]:
from pprint import pprint

pprint(proc_true_symptom_template_conversation_history_data[:2])

[[{'role': 'user', 'text': '模拟问诊,你的角色是小太医医生，有丰富的医疗问诊知识和经验。'},
  {'role': 'assistant', 'text': '您好，我是小太医，有什么可以帮到您的？'},
  {'role': 'user', 'text': '您好，声带白斑好治疗吗,是的'},
  {'role': 'assistant', 'text': '请问您发现声带白斑有多久了呢？有相关报告吗？'},
  {'role': 'user', 'text': '报告在家里'},
  {'role': 'assistant', 'text': '请问有没有声音嘶哑、咽部异物感、声带活动受限等情况呢？'},
  {'role': 'user', 'text': '我现在只能把情况跟你讲讲，晚上您要是有空我可以发给你'},
  {'role': 'assistant', 'text': '可以的啊,我们晚上也有医生在的'},
  {'role': 'user',
   'text': '就是因为嗓子哑，还伴有咳嗽，这种情况从3月份阳了以后就开始了，但是我上个月才去医院做的喉镜才茶出来,查出来'},
  {'role': 'assistant', 'text': '咳嗽目前有痰吗？'},
  {'role': 'user',
   'text': '我现在就是在家做雾化，每天2次，用的是人干扰素2a，地塞米松磷酸钠，和氯化钠，还有一个治疗食道反流的药,没痰'},
  {'role': 'assistant', 'text': '您是否有哮喘、支气管炎、鼻窦炎、扁桃体炎、反流性食管炎病史?'},
  {'role': 'user',
   'text': '当时治疗一周以后复查的喉镜有所好转，现在还在继续，每天晚上起来声音会好一点，但只要一到晚上就会比较哑，而且，现在体重长的也比较快，从用药到现在长了8斤，我已经做雾化快4周的，这个发胖跟用地塞米松有关系吗？是激素造成的吧/'},
  {'role': 'assistant',
   'text': '一般来说，雾化的用药不会引起肥胖。\n'
           '根据您的情况，以下是饮食和治疗建议：\n'
           '治疗建议：\n'
           '根据您的描述，声

In [51]:
@dataclass
class data_conf:
    Source_Seq_Token_Len=800
    Target_Seq_Token_Len=465

IGNORE_TOKEN_ID=-100

def preprocess(sources,tokenizer,system_message= "You are a helpful assistant.") -> Dict:
    roles = {"user": "<|im_start|>user", "assistant": "<|im_start|>assistant"}

    im_start = tokenizer.im_start_id
    im_end = tokenizer.im_end_id
    nl_tokens = tokenizer('\n').input_ids
    _system = tokenizer('system').input_ids + nl_tokens
    _user = tokenizer('user').input_ids + nl_tokens
    _assistant = tokenizer('assistant').input_ids + nl_tokens

    # Apply prompt templates
    input_ids, targets = [], []
    for i, source in enumerate(sources):
        ## 要求首个角色必须为User
        if roles[source[0]["role"]] != roles["user"]:
            source = source[1:]

        input_id, target = [], []
        system = [im_start] + _system + tokenizer(system_message).input_ids + [im_end] + nl_tokens
        input_id += system
        #由于都是手动添加：im_start，im_end，nl_tokens，故这三个special-token不预测。
        target += [im_start] + [IGNORE_TOKEN_ID] * (len(system)-3) + [im_end] + nl_tokens
        #print("system len: ",len(system),"target len: ",len(target))
        assert len(input_id) == len(target) ,"Not equal-1"
        for j, sentence in enumerate(source):
            role = roles[sentence["role"]]
            ## 统一处理文本，核心是信息文本，其他token[im_start，im_end，nl_tokens]手动添加，所以模型不应该预测。
            _input_id = tokenizer(role).input_ids + nl_tokens + tokenizer(sentence["text"]).input_ids + [im_end] + nl_tokens
            input_id += _input_id

            ## 按不同角色处理不同信息文本，用户角色文本不预测，重点预测助手的文本内容。
            if role == '<|im_start|>user':
                #由于都是手动添加：im_start，im_end，nl_tokens，故这三个special-token不预测。
                _target = [im_start] + [IGNORE_TOKEN_ID] * (len(_input_id)-3) + [im_end] + nl_tokens
            elif role == '<|im_start|>assistant':
                #由于都是手动添加：im_start，im_end，nl_tokens，故这三个special-token不预测。
                #只预测[assistant_text_token],但不预测[assistant+\n]
                _target = [im_start] + [IGNORE_TOKEN_ID] * len(tokenizer(role).input_ids) + \
                    _input_id[len(tokenizer(role).input_ids)+1:-2] + [im_end] + nl_tokens
            else:
                raise NotImplementedError
            target += _target
            #print("_input_id len: ",len(system),"_target len: ",len(target))
        assert len(input_id) == len(target),"Not equal-2"
        input_ids.append(input_id)
        targets.append(target)

    return dict(
        input_ids=input_ids,
        labels=targets)

In [64]:
##############
# Compact-style
##############

train_model_inputs={"input_ids": [],"labels": [],"content":[]}
all_nums=len(proc_true_symptom_template_conversation_history_data)
batch_size=2
log_steps=20
logger.info(f"run nums: {len(list(range(0,all_nums,batch_size)))} , batch_size: {batch_size}")
for idx in range(0,all_nums,batch_size):
    if idx%log_steps==0:
        logger.info("run step: {}, finished: {:.2%}".format(idx,idx/all_nums))
    sub_batch_conv_stream=proc_true_symptom_template_conversation_history_data[idx:(idx+batch_size)]
    sub_batch_train_ids=preprocess(sub_batch_conv_stream,tokenizer)
    train_model_inputs["input_ids"].extend(sub_batch_train_ids["input_ids"])
    train_model_inputs["labels"].extend(sub_batch_train_ids["labels"])
    train_model_inputs["content"].extend(sub_batch_conv_stream)


2023-12-18 16:14:36,256 - INFO : run nums: 615 , batch_size: 2
2023-12-18 16:14:36,257 - INFO : run step: 0, finished: 0.00%
2023-12-18 16:14:36,307 - INFO : run step: 20, finished: 1.63%
2023-12-18 16:14:36,353 - INFO : run step: 40, finished: 3.25%
2023-12-18 16:14:36,404 - INFO : run step: 60, finished: 4.88%
2023-12-18 16:14:36,450 - INFO : run step: 80, finished: 6.50%
2023-12-18 16:14:36,493 - INFO : run step: 100, finished: 8.13%
2023-12-18 16:14:36,537 - INFO : run step: 120, finished: 9.76%
2023-12-18 16:14:36,580 - INFO : run step: 140, finished: 11.38%
2023-12-18 16:14:36,624 - INFO : run step: 160, finished: 13.01%
2023-12-18 16:14:36,676 - INFO : run step: 180, finished: 14.63%
2023-12-18 16:14:36,724 - INFO : run step: 200, finished: 16.26%
2023-12-18 16:14:36,764 - INFO : run step: 220, finished: 17.89%
2023-12-18 16:14:36,803 - INFO : run step: 240, finished: 19.51%
2023-12-18 16:14:36,849 - INFO : run step: 260, finished: 21.14%
2023-12-18 16:14:36,894 - INFO : run ste

In [65]:
def convert_byte_token(x_token):
    if isinstance(x_token,bytes):
        try:
            x_token=x_token.decode("utf8")
        except:
            x_token=""
    return x_token

def check_train_data(tokenizer,greet_proc_result,test_idx=0):
    assert test_idx<= len(greet_proc_result["input_ids"])-1 ,"idx exceeded"
    print("Start checking ans displaying: \n")
    source_text=tokenizer.convert_ids_to_tokens(greet_proc_result["input_ids"][test_idx])
    print("source_text: ",source_text)
    target_ids_list=[item for item in greet_proc_result["labels"][test_idx] if item!=-100]
    print("target_ids   : ",target_ids_list)
    target_tokens=tokenizer.convert_ids_to_tokens(target_ids_list)
    target_tokens=[convert_byte_token(item) for item in target_tokens]
    print("target_tokens: ",target_tokens)
    target_string_list="".join(target_tokens).split("<|im_start|><|im_end|>")
    print("***"*5+"  target text: \n")
    for sub_string in target_string_list:
        sub_string=sub_string.replace("<|im_start|>","").replace("<|im_end|>","")
        sub_string=sub_string.strip()
        if sub_string!="":
            sub_string=sub_string.replace("<|im_start|>","").replace("<|im_end|>","")
            print("target_text: ",sub_string)
    print("---"*5+"  history text: \n")
    for sub_dict in greet_proc_result["content"][test_idx]:
        sub_role=sub_dict["role"]
        sub_text=sub_dict["text"]
        print("{: >9s}: {}".format(sub_role,sub_text))

In [66]:
check_train_data(tokenizer,train_model_inputs,2)

Start checking ans displaying: 

source_text:  ['<|im_start|>', b'system', b'\n', b'You', b' are', b' a', b' helpful', b' assistant', b'.', '<|im_end|>', b'\n', '<|im_start|>', b'user', b'\n', b'\xe6\xa8\xa1\xe6\x8b\x9f', b'\xe9\x97\xae', b'\xe8\xaf\x8a', b',', b'\xe4\xbd\xa0\xe7\x9a\x84', b'\xe8\xa7\x92\xe8\x89\xb2', b'\xe6\x98\xaf', b'\xe5\xb0\x8f', b'\xe5\xa4\xaa', b'\xe5\x8c\xbb', b'\xe5\x8c\xbb\xe7\x94\x9f', b'\xef\xbc\x8c', b'\xe6\x9c\x89', b'\xe4\xb8\xb0\xe5\xaf\x8c\xe7\x9a\x84', b'\xe5\x8c\xbb\xe7\x96\x97', b'\xe9\x97\xae', b'\xe8\xaf\x8a', b'\xe7\x9f\xa5\xe8\xaf\x86', b'\xe5\x92\x8c', b'\xe7\xbb\x8f\xe9\xaa\x8c', b'\xe3\x80\x82', '<|im_end|>', b'\n', '<|im_start|>', b'assistant', b'\n', b'\xe6\x82\xa8\xe5\xa5\xbd', b'\xef\xbc\x8c', b'\xe6\x88\x91\xe6\x98\xaf', b'\xe5\xb0\x8f', b'\xe5\xa4\xaa', b'\xe5\x8c\xbb', b'\xef\xbc\x8c', b'\xe6\x9c\x89\xe4\xbb\x80\xe4\xb9\x88', b'\xe5\x8f\xaf\xe4\xbb\xa5', b'\xe5\xb8\xae', b'\xe5\x88\xb0', b'\xe6\x82\xa8\xe7\x9a\x84', b'\xef\xbc\x9f', '<

In [63]:
def show_data_distribution(all_conversation_data_result_list):
    #### statistic conversation string length
    conv_seq_len=[]
    conv_turns_lens_result=[]
    for idx in range(len(all_conversation_data_result_list["input_ids"])):
        sub_prompt_len=len(all_conversation_data_result_list["input_ids"][idx])
        sub_output_len=len(all_conversation_data_result_list["labels"][idx])
        assert sub_prompt_len==sub_output_len ,"异常" +str(idx)+"||".join([item["text"] for item in all_conversation_data_result_list["content"][idx][-2:]])
        conv_seq_len.append(sub_prompt_len)
        conv_turns_lens_result.append(len(all_conversation_data_result_list["content"][idx])//2)
        if sub_prompt_len>1900:
        #if sub_output_len>200:
            print('output: ',all_conversation_data_result_list[idx]["content"][-1])

    print(pd.Series(conv_seq_len).describe(percentiles=[0.01,0.1,0.2,0.25,0.5,0.75,0.8,0.9,0.99,0.9999]))
    print(pd.Series(conv_turns_lens_result).describe(percentiles=[0.01,0.1,0.2,0.25,0.5,0.75,0.8,0.9,0.99,0.9999]))

count     100.000000
mean      540.010000
std       101.175265
min       309.000000
1%        323.850000
10%       408.900000
20%       463.800000
25%       477.750000
50%       547.000000
75%       602.500000
80%       614.000000
90%       644.000000
99%       795.690000
99.99%    863.316900
max       864.000000
dtype: float64
count     100.000000
mean        6.720000
std         1.477645
min         4.000000
1%          4.000000
10%         5.000000
20%         5.000000
25%         6.000000
50%         7.000000
75%         8.000000
80%         8.000000
90%         9.000000
99%        10.010000
99.99%     10.990100
max        11.000000
dtype: float64


In [67]:
show_data_distribution(train_model_inputs)


count     1230.000000
mean       516.476423
std        100.359358
min        218.000000
1%         315.000000
10%        400.900000
20%        431.000000
25%        445.000000
50%        513.000000
75%        581.000000
80%        599.000000
90%        639.100000
99%        790.710000
99.99%     928.871800
max        936.000000
dtype: float64
count     1230.000000
mean         6.487805
std          1.592118
min          3.000000
1%           4.000000
10%          5.000000
20%          5.000000
25%          5.000000
50%          6.000000
75%          7.000000
80%          8.000000
90%          9.000000
99%         11.000000
99.99%      12.000000
max         12.000000
dtype: float64


In [68]:
##################################
## Save-Data-16
##################################

save_mode=True

if save_mode:
    save_file="data/Qwen_train_data_d1230_s936_1218.json"

    with open(save_file,'w+') as f :
        for sub_input_ids,sub_labels,sub_cont in zip(train_model_inputs["input_ids"],
                                                     train_model_inputs["labels"],
                                                     train_model_inputs["content"]):
            sub_data=json.dumps({"input_ids":sub_input_ids,"labels":sub_labels,"content":sub_cont},ensure_ascii=False)+'\n'
            f.write(sub_data)

    logger.info(f'save file: {save_file}')


2023-12-18 16:15:49,153 - INFO : save file: data/Qwen_train_data_d1230_s936_1218.json
